In [ ]:
# conda activate py38
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
cv2.__version__

In [ ]:
# Super Resolution
path = "ESPCN/ESPCN_x4.pb"
sr = cv2.dnn_superres.DnnSuperResImpl_create()
sr.readModel(path)
sr.setModel("espcn",4)

In [ ]:
sift = cv2.xfeatures2d.SIFT_create(1000)

In [ ]:
prev_path = "previous_img/"
cur_path = "current_img/"
prev_img = []
cur_img = []
prev_img_list = os.listdir(prev_path)
print(prev_img_list)
cur_img_list = os.listdir(cur_path)
print(cur_img_list)
#print('Total Classes Detected', len(pre_img_list))
class_names = []

for cl in prev_img_list:
    img = cv2.imread(f'{prev_path}/{cl}',0)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = sr.upsample(img) # Super Resolution
    img = cv2.pyrUp(img)
    img = cv2.pyrUp(img)
    prev_img.append(img)
    class_names.append(os.path.splitext(cl)[0])

for cl in cur_img_list:
    img = cv2.imread(f'{cur_path}/{cl}',0)
    img = sr.upsample(img) # Super Resolution
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.pyrUp(img)
    img = cv2.pyrUp(img)
    cur_img.append(img)
    
print(class_names)

In [ ]:
def findDes(images):
    desList=[]
    for img in images:
        kp, des = sift.detectAndCompute(img,None)
        desList.append(des)
    return desList

In [ ]:
def findID(img, prev_desList, thres):
    cur_kpn, cur_des = sift.detectAndCompute(img, None)
    
    bf = cv2.BFMatcher()
    matchList = []
    finalVal = -1
    try:
        for p_des in prev_desList:
            matches = bf.knnMatch(p_des, cur_des, k=2)
            good = []
            for m, n in matches:
                if m.distance <0.75 *n.distance:
                    good.append([m])
            #print(len(good))
            matchList.append(len(good))
        #print(matchList)
    except:
        print("pass")
        pass

    if len(matchList)!=0:
        if max(matchList) > thres:
            finalVal = matchList.index(max(matchList))

    return finalVal

In [ ]:
print(prev_img_list)

In [ ]:
# giving img_index=0~4 to test ID recognition
img_index = 0
prev_desList = findDes(prev_img)
rec_id = findID(cur_img[img_index], prev_desList, 2)

print("bring current frame object: " + cur_img_list[img_index] + " to find objcet on the previous frame")
plt.imshow(cur_img[img_index])
plt.show()
if rec_id != -1:
    regc = class_names[rec_id]
    print("recognized: " + regc)
    plt.imshow(prev_img[class_names.index(regc)])
    plt.show()
else:
    print("bring current frame object:" + cur_img_list[img_index] + " to find objcet on the previous frame")
    print("couldn't find this object on the previous frame")